使用R语言学习MLE，光会调包不行，要懂统计也要会Code。

In [4]:
rosen <- function(x) {
  100 * (x[2] - x[1]^2)^2 + (1 - x[1])^2
}

result <- optim(c(-1.2, 1), rosen, hessian = TRUE)

In [5]:
summary(result)

            Length Class  Mode   
par         2      -none- numeric
value       1      -none- numeric
counts      2      -none- numeric
convergence 1      -none- numeric
message     0      -none- NULL   
hessian     4      -none- numeric

In [3]:
result$par
result$value

[1] 1.000260 1.000506

[1] 8.825241e-08

In [6]:
result$hessian

802.4228,-400.1041
-400.1041,200.0000


In [7]:
library(maxLik)

Loading required package: miscTools


Please cite the 'maxLik' package as:
Henningsen, Arne and Toomet, Ott (2011). maxLik: A package for maximum likelihood estimation in R. Computational Statistics 26(3), 443-458. DOI 10.1007/s00180-010-0217-1.

If you have questions, suggestions, or comments regarding the 'maxLik' package, please use a forum or 'tracker' at maxLik's R-Forge site:
https://r-forge.r-project.org/projects/maxlik/



In [52]:
set.seed(123)
x <- rnorm(200, mean = 5, sd = 2)  # 真正的参数是 mu = 5, sigma = 2
N <- length(x)

# 定义对数似然函数
loglik <- function(param) {
  mu <- param[1]
  sigma <- param[2]
  # 使用 dnorm 的 log=TRUE 参数计算每个观测的 log-likelihood
  sum(dnorm(x, mean = mu, sd = sigma, log = TRUE))
}

# 使用 maxLik 进行估计
library(maxLik)
result <- maxLik(logLik = loglik,
                 start = c(mu = 0, sigma = 1),
                 control = list(iterlim = 3000, printLevel = 1))

# 输出结果
summary(result)

Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, sd = sigma, log = TRUE):
"NaNs produced"
Warning message in dnorm(x, mean = mu, s

--------------
successive function values within relative tolerance limit (reltol) 
10  iterations
estimate: 4.982859 1.881598 
Function value: -410.212 


--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 10 iterations
Return code 8: successive function values within relative tolerance limit (reltol)
Log-Likelihood: -410.212 
2  free parameters
Estimates:
      Estimate Std. error t value Pr(> t)    
mu     4.98286    0.13337   37.36  <2e-16 ***
sigma  1.88160    0.09391   20.04  <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

In [53]:
coef(result)

mu    sigma 
4.982859 1.881598

In [56]:
logLik(result)

[1] -410.212
attr(,"df")
[1] 2

In [57]:
AIC(result)

[1] 824.424
attr(,"df")
[1] 2

In [58]:
vcov(result)

,mu,sigma
mu,1.778793e-02,-2.674877e-05
sigma,-2.674877e-05,8.818179e-03


In [59]:
result$hessian

,mu,sigma
mu,-56.2181413,-0.1705303
sigma,-0.1705303,-113.4026206


In [64]:
# 线性回归的MLE
# 生成数据
set.seed(123)
n <- 100
x <- rnorm(n)
y <- 2 + 3 * x + rnorm(n)
data <- data.frame(x = x, y = y)

# 定义对数似然函数
loglik <- function(param) {
  beta0 <- param[1]
  beta1 <- param[2]
  sigma <- exp(param[3])  # 确保 sigma 是正的
  # 使用 dnorm 的 log=TRUE 参数计算每个观测的 log-likelihood
  sum(dnorm(data$y, mean = beta0 + beta1 * data$x, sd = sigma, log = TRUE))
}
# 使用 maxLik 进行估计
result <- maxLik(logLik = loglik,
                 start = c(beta0 = 0, beta1 = 0, log_sigma = 0),
                 control = list(iterlim = 3000, printLevel = 1))
# 输出结果
library(broom)
tidy(result)

--------------
gradient close to zero (gradtol) 
7  iterations
estimate: 1.897197 2.947528 -0.03982402 
Function value: -137.9115 


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
beta0,1.89719695,0.09655366,19.6491464,5.879330e-86
beta1,2.94752839,0.10588322,27.8375401,1.524523e-170
log_sigma,-0.03982402,0.07073017,-0.5630415,5.734066e-01


In [65]:
# 使用glm作为对比
result_glm <- glm(y ~ x, data = data)
tidy(result_glm)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.897197,0.09755118,19.44822,2.046638e-35
x,2.947528,0.10687862,27.57828,5.575170e-48
